In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("dataport-export_gas_oct2015-mar2016.csv")

In [3]:
df.head()

,localminute,dataid,meter_value
0,2015-10-01 00:00:10-05,739,88858
1,2015-10-01 00:00:13-05,8890,197164
2,2015-10-01 00:00:20-05,6910,179118
3,2015-10-01 00:00:22-05,3635,151318
4,2015-10-01 00:00:22-05,1507,390354


In [4]:
df.dtypes

localminute    object
dataid          int64
meter_value     int64
dtype: object

In [5]:
df.groupby('dataid')['meter_value'].describe()

,count,mean,std,min,25%,50%,75%,max
dataid,,,,,,,,
35,11872.0,98799.541779,3585.696685,93470.0,95402.0,98397.0,102309.0,104692.0
44,1549.0,168857.252421,1621.907607,165674.0,167868.0,168188.0,169672.0,175070.0
77,10683.0,61162.735748,1655.197037,58832.0,59790.0,60742.0,62590.0,64652.0
94,36335.0,126452.122086,8260.652894,116642.0,117894.0,124482.0,134538.0,140358.0
114,2597.0,136067.471698,6187.644319,128294.0,129930.0,134774.0,143072.0,145298.0
...,...,...,...,...,...,...,...,...
9729,12361.0,129427.883019,5794.697171,122186.0,123380.0,128152.0,135684.0,138146.0
9766,2282.0,171690.504820,7682.217141,158512.0,161656.0,176406.0,177428.0,179902.0
9849,2741.0,35566.648668,1675.226417,33632.0,34070.0,35172.0,36632.0,39292.0


In [6]:
df['localminute'].head()

0    2015-10-01 00:00:10-05
1    2015-10-01 00:00:13-05
2    2015-10-01 00:00:20-05
3    2015-10-01 00:00:22-05
4    2015-10-01 00:00:22-05
Name: localminute, dtype: object

As we can see, the localminute column containing the timestamp for each meter reading is:
1. Incorrectly formatted - the the "-05" or "-06" at the end is timezone data, indicating when Daylight Savings Time begins and ends. We can extract this to create a new feature
2. Not in the datetime datatype

In [7]:
df['DST'] = pd.to_numeric(df['localminute'].str[-2:]) == 5
df

,localminute,dataid,meter_value,DST
0,2015-10-01 00:00:10-05,739,88858,True
1,2015-10-01 00:00:13-05,8890,197164,True
2,2015-10-01 00:00:20-05,6910,179118,True
3,2015-10-01 00:00:22-05,3635,151318,True
4,2015-10-01 00:00:22-05,1507,390354,True
...,...,...,...,...
1584818,2016-03-31 23:59:14.336743-05,2129,201726,True
1584819,2016-03-31 23:59:17.427165-05,2945,161232,True
1584820,2016-03-31 23:59:35.370782-05,9729,138146,True
1584821,2016-03-31 23:59:47.816286-05,5129,166488,True


In [8]:
df['localminute'] = df['localminute'].str[:-3]
df['localminute'] = pd.to_datetime(df['localminute'], format='mixed').dt.strftime('%Y-%m-%d %H:%M:%S')
df['localminute'] = pd.to_datetime(df['localminute'])

In [9]:
df

,localminute,dataid,meter_value,DST
0,2015-10-01 00:00:10,739,88858,True
1,2015-10-01 00:00:13,8890,197164,True
2,2015-10-01 00:00:20,6910,179118,True
3,2015-10-01 00:00:22,3635,151318,True
4,2015-10-01 00:00:22,1507,390354,True
...,...,...,...,...
1584818,2016-03-31 23:59:14,2129,201726,True
1584819,2016-03-31 23:59:17,2945,161232,True
1584820,2016-03-31 23:59:35,9729,138146,True
1584821,2016-03-31 23:59:47,5129,166488,True


In [10]:
df.dtypes

localminute    datetime64[ns]
dataid                  int64
meter_value             int64
DST                      bool
dtype: object

In [11]:
df['dataid'].value_counts()

dataid
2034    75991
6910    69349
484     44034
5814    42424
5810    42234
        ...  
7566       32
9620       23
4671       21
6101        3
4874        2
Name: count, Length: 157, dtype: int64

Let's plot a time series graph for each dataid and try to see if there are any outliers, anomalies, or any patterns in the data. The meter values we have currently measure cumulative consumption and not marginal consumption.

In [12]:
df = df.set_index(df['dataid'])
df = df.drop(columns='dataid')

In [13]:
df.index.value_counts()

dataid
2034    75991
6910    69349
484     44034
5814    42424
5810    42234
        ...  
7566       32
9620       23
4671       21
6101        3
4874        2
Name: count, Length: 157, dtype: int64

In [14]:
df = df.drop([7566, 9620, 4671, 6101, 4874, 9160, 2755, 2946, 2814, 5545, 2645, 6685, 1403])
df.index.value_counts()

dataid
2034    75991
6910    69349
484     44034
5814    42424
5810    42234
        ...  
8059      529
5658      493
3036      336
9600      330
8703      258
Name: count, Length: 144, dtype: int64

I've gone ahead and dropped all the dataids that have less than 250 readings. I've seen later that these end up being quite problematic and irregular, so I've decided that they don't have enough data to be included in our investigation

In [15]:
df = df.reset_index()
df

,dataid,localminute,meter_value,DST
0,739,2015-10-01 00:00:10,88858,True
1,8890,2015-10-01 00:00:13,197164,True
2,6910,2015-10-01 00:00:20,179118,True
3,3635,2015-10-01 00:00:22,151318,True
4,1507,2015-10-01 00:00:22,390354,True
...,...,...,...,...
1584128,2129,2016-03-31 23:59:14,201726,True
1584129,2945,2016-03-31 23:59:17,161232,True
1584130,9729,2016-03-31 23:59:35,138146,True
1584131,5129,2016-03-31 23:59:47,166488,True


In [16]:
df = df.sort_values(by=['dataid', 'localminute'])

As we can see, the data is irregularly sampled, which is inconvenient to handle and creates a lot of uneccesary noise, especially as there is barely any difference at small intervals. Therefore we will extract the marginal consumption per hour from the cumulative readings using resampling.

In [17]:
# for dataid, group in df.groupby('dataid'):
#     fig, ax = plt.subplots(figsize=(10,5))
#     plt.plot(df[df['dataid'] == dataid].set_index("localminute")['meter_value'].diff())
#     ax.set_title(f'meter value by time for dataid {dataid}')
#     plt.show()

In [18]:
marginal_consumption = []
for dataid, group in df.groupby('dataid'):
    meter_value_diff = group['meter_value'].diff()
    marginal_consumption.append(meter_value_diff)


In [19]:
marginal_consumption = pd.concat(marginal_consumption)
df['marginal_consumption'] = marginal_consumption
df['marginal_consumption']

83          NaN
244         0.0
723         0.0
790         0.0
829         0.0
           ... 
1560987     0.0
1561582     2.0
1566451    48.0
1576619    68.0
1580897    72.0
Name: marginal_consumption, Length: 1584133, dtype: float64

In [20]:
df['marginal_consumption'] = df['marginal_consumption'].fillna(0)

In [21]:
df

,dataid,localminute,meter_value,DST,marginal_consumption
83,35,2015-10-01 00:14:44,93470,True,0.0
244,35,2015-10-01 00:42:34,93470,True,0.0
723,35,2015-10-01 02:02:37,93470,True,0.0
790,35,2015-10-01 02:12:38,93470,True,0.0
829,35,2015-10-01 02:20:36,93470,True,0.0
...,...,...,...,...,...
1560987,9982,2016-03-28 09:25:54,119734,True,0.0
1561582,9982,2016-03-28 11:29:41,119736,True,2.0
1566451,9982,2016-03-29 04:38:38,119784,True,48.0
1576619,9982,2016-03-30 17:46:42,119852,True,68.0


In [22]:
resampled_consumption = []
dataid_column = []
table = []

for dataid, group in df.groupby('dataid'):
    resample = group.resample('h', on='localminute')['marginal_consumption'].sum()
    resampled_consumption.append(resample)
    for i in range(len(resample)):
        dataid_column.append(dataid)

In [23]:
consumption_series = pd.concat(resampled_consumption)
consumption_series

localminute
2015-10-01 00:00:00     0.0
2015-10-01 01:00:00     0.0
2015-10-01 02:00:00     0.0
2015-10-01 03:00:00     0.0
2015-10-01 04:00:00     0.0
                       ... 
2016-03-31 07:00:00     0.0
2016-03-31 08:00:00     0.0
2016-03-31 09:00:00     0.0
2016-03-31 10:00:00     0.0
2016-03-31 11:00:00    72.0
Name: marginal_consumption, Length: 591478, dtype: float64

In [24]:
df = pd.DataFrame(consumption_series)

In [25]:
len(dataid_column)

591478

In [26]:
df['dataid'] = dataid_column
df

,marginal_consumption,dataid
localminute,,
2015-10-01 00:00:00,0.0,35
2015-10-01 01:00:00,0.0,35
2015-10-01 02:00:00,0.0,35
2015-10-01 03:00:00,0.0,35
2015-10-01 04:00:00,0.0,35
...,...,...
2016-03-31 07:00:00,0.0,9982
2016-03-31 08:00:00,0.0,9982
2016-03-31 09:00:00,0.0,9982


In [27]:
df['DST'] = df.index.map( lambda x: 1 if x < pd.to_datetime('2015-11-01 01:00:00', format='mixed') or x > pd.to_datetime('2016-03-13 02:00:00', format='mixed') else 0 )

In [28]:
df

,marginal_consumption,dataid,DST
localminute,,,
2015-10-01 00:00:00,0.0,35,1
2015-10-01 01:00:00,0.0,35,1
2015-10-01 02:00:00,0.0,35,1
2015-10-01 03:00:00,0.0,35,1
2015-10-01 04:00:00,0.0,35,1
...,...,...,...
2016-03-31 07:00:00,0.0,9982,1
2016-03-31 08:00:00,0.0,9982,1
2016-03-31 09:00:00,0.0,9982,1


Now we have our new dataset resampled by the hour, with marginal consumption extracted. Let's have a look at the time series graphs now.

It seems there are outliers present that could hamper our model performance. Let us try and remove them. Firstly, let's start with the ones with a large negative marginal consumption. Since our meter value data is cumulative, the marginal consumption cannot be negative

In [29]:
df = df[~df['dataid'].isin(df[df['marginal_consumption'] < -50]['dataid'].unique())]   

In [30]:
len(df[df['marginal_consumption'] < -50]['dataid'].unique())

0

In [31]:
len(df['dataid'].unique())

129

In [32]:
extreme = df[df['marginal_consumption'] > 5000]['dataid'].unique()

In [33]:
len(df[df['dataid'] == 4874])

0

In [34]:
df['marginal_consumption'].min()

0.0

In [35]:
len(df[df['marginal_consumption'] > 1000]['dataid'].unique())

22

In [36]:
df['marginal_consumption'].max()

13982.0

In [37]:
removed = []
for dataid, group in df.groupby('dataid'): 
    ub = group['marginal_consumption'].quantile(0.99)
    # outliers = group['marginal_consumption'][group['marginal_consumption'] > ub]
    group.loc[group['marginal_consumption'] > ub] = np.nan
    group = group.interpolate()
    removed.append(group['marginal_consumption'])

In [38]:
df['marginal_consumption'] = pd.concat(removed)
df['marginal_consumption'].isna().sum()

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/3578771186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marginal_consumption'] = pd.concat(removed)


0

In [39]:
# for dataid, group in df.groupby('dataid'): 
#     plt.hist(group['marginal_consumption'])
#     plt.show()

In [40]:
from scipy import stats

In [41]:
df.reset_index(inplace=True)

In [42]:
df["marginal_diff"] = df["marginal_consumption"].diff()
df.fillna(0, inplace=True)
df

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/1795095638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["marginal_diff"] = df["marginal_consumption"].diff()
/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/1795095638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


,localminute,marginal_consumption,dataid,DST,marginal_diff
0,2015-10-01 00:00:00,0.0,35,1,0.0
1,2015-10-01 01:00:00,0.0,35,1,0.0
2,2015-10-01 02:00:00,0.0,35,1,0.0
3,2015-10-01 03:00:00,0.0,35,1,0.0
4,2015-10-01 04:00:00,0.0,35,1,0.0
...,...,...,...,...,...
525616,2016-02-08 12:00:00,0.0,9956,0,0.0
525617,2016-02-08 13:00:00,0.0,9956,0,0.0
525618,2016-02-08 14:00:00,0.0,9956,0,0.0
525619,2016-02-08 15:00:00,0.0,9956,0,0.0


In [43]:
df['localminute'].dt.year

0         2015
1         2015
2         2015
3         2015
4         2015
          ... 
525616    2016
525617    2016
525618    2016
525619    2016
525620    2016
Name: localminute, Length: 525621, dtype: int32

In [44]:
df['week'] = df['localminute'].dt.isocalendar().week
df['week']

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/2349163691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week'] = df['localminute'].dt.isocalendar().week


0         40
1         40
2         40
3         40
4         40
          ..
525616     6
525617     6
525618     6
525619     6
525620     6
Name: week, Length: 525621, dtype: UInt32

In [45]:
standardized = []
for dataid, group in df.groupby('dataid'):
    weekly_variance = group.groupby(group['localminute'].dt.isocalendar().week)['marginal_diff'].std()
    mapped_weekly_variance = group['localminute'].map(lambda x: weekly_variance.loc[x.week])
    group['marginal_diff'] = np.where(mapped_weekly_variance != 0, group['marginal_diff'] / mapped_weekly_variance, 0)
    standardized.append(group['marginal_diff'])
    # group.set_index('localminute')['marginal_diff'].plot(figsize=(15,5))
    # plt.xlabel('Date')
    # plt.ylabel('Standardized beer production rate')
    # plt.grid()
    # plt.show()

In [46]:
df['marginal_diff'] = pd.concat(standardized)
df['marginal_diff']

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/2381088770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marginal_diff'] = pd.concat(standardized)


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
525616    0.0
525617    0.0
525618    0.0
525619    0.0
525620    0.0
Name: marginal_diff, Length: 525621, dtype: float64

In [47]:
df.isna().sum()

localminute             0
marginal_consumption    0
dataid                  0
DST                     0
marginal_diff           0
week                    0
dtype: int64

In [48]:
df['marginal_diff'].skew()

0.09241104017679351

In [49]:
df['year'] = df['localminute'].dt.year
df['month'] = df['localminute'].dt.month
df['day'] = df['localminute'].dt.day
df['hour'] = df['localminute'].dt.hour
df

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/3429711195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['localminute'].dt.year
/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/3429711195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['localminute'].dt.month
/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/3429711195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,localminute,marginal_consumption,dataid,DST,marginal_diff,week,year,month,day,hour
0,2015-10-01 00:00:00,0.0,35,1,0.0,40,2015,10,1,0
1,2015-10-01 01:00:00,0.0,35,1,0.0,40,2015,10,1,1
2,2015-10-01 02:00:00,0.0,35,1,0.0,40,2015,10,1,2
3,2015-10-01 03:00:00,0.0,35,1,0.0,40,2015,10,1,3
4,2015-10-01 04:00:00,0.0,35,1,0.0,40,2015,10,1,4
...,...,...,...,...,...,...,...,...,...,...
525616,2016-02-08 12:00:00,0.0,9956,0,0.0,6,2016,2,8,12
525617,2016-02-08 13:00:00,0.0,9956,0,0.0,6,2016,2,8,13
525618,2016-02-08 14:00:00,0.0,9956,0,0.0,6,2016,2,8,14
525619,2016-02-08 15:00:00,0.0,9956,0,0.0,6,2016,2,8,15


In [50]:
features = ['marginal_diff', 'DST', 'week', 'year', 'month', 'day', 'hour', 'dataid']

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
train_data, test_val_data = train_test_split(df[features], test_size=0.4, random_state=42, shuffle=False)
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42, shuffle=False)

In [53]:
X_train = train_data
y_train = train_data['marginal_diff'].copy()
X_test = test_data
y_test = test_data['marginal_diff'].copy()
X_val = val_data
y_val = val_data['marginal_diff'].copy()

In [54]:
# Step 3: Define the missing value rate
missing_rate = 0.8  # Introduce NaNs in 80% of the marginal_diff data

# Step 4: Randomly assign NaNs in X_train's marginal_diff column
n_train_missing = int(len(X_train) * missing_rate)
train_indices = np.random.choice(X_train.index, n_train_missing, replace=False)
X_train.loc[train_indices, 'marginal_diff'] = np.nan

In [55]:
# Introduce NaNs in X_test's marginal_diff column
n_test_missing = int(len(X_test) * missing_rate)
test_indices = np.random.choice(X_test.index, n_test_missing, replace=False)
X_test.loc[test_indices, 'marginal_diff'] = np.nan

In [56]:
n_val_missing = int(len(X_val) * missing_rate)
val_indices = np.random.choice(X_val.index, n_val_missing, replace=False)
X_val.loc[val_indices, 'marginal_diff'] = np.nan

In [57]:
mask_train = np.where(np.isnan(X_train['marginal_diff']), 0, 1)  # Mask based on original data shape
mask_test = np.where(np.isnan(X_test['marginal_diff']), 0, 1)
mask_val = np.where(np.isnan(X_val['marginal_diff']), 0, 1)

In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_filled = pd.DataFrame(X_train).fillna(method='ffill')
scaled_X_train = scaler.fit_transform(X_train_filled)

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/3897026729.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train_filled = pd.DataFrame(X_train).fillna(method='ffill')


In [59]:
X_test_filled = pd.DataFrame(X_test).fillna(method='ffill')
scaled_X_test = scaler.transform(X_test_filled)
X_val_filled = pd.DataFrame(X_val).fillna(method='ffill')
scaled_X_val = scaler.transform(X_val_filled)

/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/2740955589.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test_filled = pd.DataFrame(X_test).fillna(method='ffill')
/var/folders/tn/8lbbw7dn3f3b367scfqs7w3h0000gp/T/ipykernel_18285/2740955589.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_val_filled = pd.DataFrame(X_val).fillna(method='ffill')


In [60]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

In [61]:
sequence_length = 24
X_train_seq = create_sequences(scaled_X_train, sequence_length)
X_test_seq = create_sequences(scaled_X_test, sequence_length)
X_val_seq = create_sequences(scaled_X_val, sequence_length)
y_train_seq = create_sequences(y_train, sequence_length)
y_test_seq = create_sequences(y_test, sequence_length)
y_val_seq = create_sequences(y_val, sequence_length)

In [62]:
mask_train_seq = create_sequences(mask_train, sequence_length)
mask_test_seq = create_sequences(mask_test, sequence_length)
mask_val_seq = create_sequences(mask_val, sequence_length)

In [63]:
import torch

In [64]:
X_train_tensor = torch.from_numpy(X_train_seq).float()
X_test_tensor = torch.from_numpy(X_test_seq).float()
X_val_tensor = torch.from_numpy(X_val_seq).float()
mask_train_tensor = torch.from_numpy(mask_train_seq).float()
mask_test_tensor = torch.from_numpy(mask_test_seq).float()
mask_val_tensor = torch.from_numpy(mask_val_seq).float()

In [65]:
y_train_tensor = torch.from_numpy(y_train_seq).float()
y_test_tensor = torch.from_numpy(y_test_seq).float()
y_val_tensor = torch.from_numpy(y_val_seq).float()

In [66]:
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [67]:
train_dataset = TensorDataset(X_train_tensor, mask_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, mask_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [68]:
class LSTMImputer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMImputer, self).__init__()
        self.lstm = nn.LSTM(input_size + 1, hidden_size, num_layers=num_layers, batch_first=True)  # +1 for mask channel
        self.fc = nn.Linear(hidden_size, 1)  # Change to output a single value

    def forward(self, x, mask):
        mask = mask.unsqueeze(-1)  # Change shape from (batch_size, seq_length) to (batch_size, seq_length, 1)
        x_masked = torch.cat((x, mask), dim=2)  # Concatenate along feature dimension
        
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        
        out, _ = self.lstm(x_masked, (h0, c0))
        imputed = self.fc(out)  # Output imputed values
        
        return imputed

In [69]:
input_size = X_train_tensor.shape[2]
hidden_size = 64            
num_layers = 2

In [70]:
model = LSTMImputer(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss(reduction='none')

In [71]:
X_train_tensor.shape

torch.Size([315349, 24, 8])

In [72]:
y_train_tensor.shape

torch.Size([315349, 24])

In [73]:
mask_train_tensor.shape

torch.Size([315349, 24])

In [74]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    
    for inputs, mask, targets in train_loader:
        optimizer.zero_grad()  # Clear gradients
        
        outputs = model(inputs.float(), mask.float())  # Forward pass
        
        loss = criterion(outputs.squeeze(), targets.squeeze())
        
        masked_loss = loss * mask.squeeze()  
        
        masked_loss_valid_entries = masked_loss[masked_loss > 0] 
        
        if len(masked_loss_valid_entries) > 0:
            final_loss = masked_loss_valid_entries.mean()  
            
            final_loss.backward()  
            optimizer.step()       

    # Validation phase after each epoch
    model.eval()   # Set model to evaluation mode
    val_loss_total = 0 
    num_val_entries = 0 

    with torch.no_grad():  # Disable gradient calculation for validation
        for inputs_val, mask_val, targets_val in val_loader:
            outputs_val = model(inputs_val.float(), mask_val.float())  
            
            loss_val = criterion(outputs_val.squeeze(), targets_val.squeeze())
            
            masked_loss_val = loss_val * mask_val.squeeze()  
            
            masked_loss_valid_entries_val = masked_loss_val[masked_loss_val > 0]  
            
            if len(masked_loss_valid_entries_val) > 0:
                val_loss_total += masked_loss_valid_entries_val.sum().item() 
                num_val_entries += len(masked_loss_valid_entries_val)  

    if num_val_entries > 0:
        average_val_loss = val_loss_total / num_val_entries 
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {final_loss.item():.4f}, Validation Loss: {average_val_loss:.4f}')
    else:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {final_loss.item():.4f}, Validation Loss: No valid entries')

Epoch [1/100], Train Loss: nan, Validation Loss: nan


KeyboardInterrupt: 

In [ ]:
# Evaluation on test data
model.eval()  # Set the model to evaluation mode
test_loss = 0
num_valid_entries = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, mask, targets in DataLoader(TensorDataset(X_test_tensor, mask_test_tensor, y_test_tensor), batch_size=32):
        outputs = model(inputs.float(), mask.float())  # Forward pass
        
        # Compute raw loss against original targets
        loss = criterion(outputs.squeeze(), targets.squeeze())
        
        # Apply mask to loss to ignore NaNs in targets
        masked_loss = loss * mask.squeeze()  # This will zero out losses for masked (NaN) targets
        
        masked_loss_valid_entries = masked_loss[masked_loss > 0]  # Keep only valid losses
        
        if len(masked_loss_valid_entries) > 0:
            test_loss += masked_loss_valid_entries.sum().item()  # Sum of valid losses
            num_valid_entries += len(masked_loss_valid_entries)  # Count valid entries

# Calculate average loss over valid entries
if num_valid_entries > 0:
    average_test_loss = test_loss / num_valid_entries
    print(f'Average Test Loss: {average_test_loss:.4f}')
else:
    print('No valid entries found in test data.')


In [ ]:
with torch.no_grad():
    predictions = model(X_test_tensor.float(), mask_test_tensor.float()).squeeze()
    print("Predictions:", predictions[:10].numpy())
    print("Actual Values:", y_test_tensor[:10].numpy())